In [1]:
import torch
from pathlib import Path
from datetime import datetime, timedelta
import time
import cv2
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import db

# Firebase 서비스 계정 키 파일 경로
cred = credentials.Certificate("fireserver-63298-firebase-adminsdk-l0ret-8e04767a64.json")

# Firebase 초기화
firebase_admin.initialize_app(cred, {
    'storageBucket': 'fireserver-63298.appspot.com',
    'databaseURL': 'https://fireserver-63298-default-rtdb.firebaseio.com/'
})

# Firebase Storage 초기화
bucket = storage.bucket()

# Firebase Realtime Database 초기화
firebase_db = db.reference('Detected')


In [3]:
# YOLOv5 모델 로드
model_path = Path("C:/Users/kelld/fire/Yolov5-Fire-Detection/yolov5/runs/train/exp9/weights/best.pt")
model = torch.hub.load("ultralytics/yolov5", "custom", path=model_path)


# 웹캠 열기
cap = cv2.VideoCapture(0)

# 로컬 저장을 위한 코덱 및 VideoWriter 객체 정의
fourcc = cv2.VideoWriter_fourcc(*'XVID')

current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # ':'를 '-'로 수정
output_file_path = f"C:/Users/kelld/fire/Yolov5-Fire-Detection/CCTVvideo/video/recorded_{current_datetime}.avi"

output_video = cv2.VideoWriter(output_file_path, fourcc, 20.0, (640, 480))  # 변수명 오타 수정

recording = False
recording_start_time = None
recorded_frames = []

while True:
    ret, frame = cap.read()

    if not ret:
        break

    img = frame[..., ::-1]  # BGR 이미지를 RGB로 변환
    results = model(img)
    pred = results.pred[0]

    object_detected = False

    for det in pred:
        bbox = det[:4].cpu().numpy().astype(int)
        score = det[4].cpu().numpy()
        class_id = int(det[5].cpu().numpy())
        class_name = "fire"

        if score > 0.5:
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name}: {score:.2f}", (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            object_detected = True

    if object_detected and not recording:
        recording_start_time = time.time()
        recording = True
        print("화재가 감지되어 녹화가 시작되었습니다.")
        
        # Firebase Realtime Database에 웹캠 상태 업데이트
        firebase_db.update({"webcam_status": "화재감지"})

    if recording:
        output_video.write(frame)

        if (time.time() - recording_start_time) >= 10:
            recording = False
            print("녹화가 종료되었습니다.")
            print("녹화된 비디오 파일:", output_file_path)
            
            # Firebase Realtime Database에 웹캠 상태 업데이트
            firebase_db.update({"webcam_status": "녹화종료"})

            # Firebase Storage에 녹화된 동영상 업로드
            destination_blob_name = f"recorded_videos/recorded_{current_datetime}.avi"
            blob = bucket.blob(destination_blob_name)
            blob.upload_from_filename(output_file_path)
            print("동영상을 Firebase Storage에 업로드했습니다:", destination_blob_name)
            
            # Firebase Storage에 업로드된 동영상의 URL 가져오기
            video_url = blob.generate_signed_url(datetime.now() + timedelta(days=365), method='GET')
            
            # Realtime Database에 URL 저장
            firebase_db.push({
                "video_url": video_url,
                "timestamp_webcam": current_datetime + " CAM1"
            })

    cv2.imshow("CAM1", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
output_video.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\kelld/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-26 Python-3.9.16 torch-1.12.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


화재가 감지되어 녹화가 시작되었습니다.
녹화가 종료되었습니다.
녹화된 비디오 파일: C:/Users/kelld/fire/Yolov5-Fire-Detection/CCTVvideo/video/recorded_2023-11-07_12-24-45.avi
동영상을 Firebase Storage에 업로드했습니다: recorded_videos/recorded_2023-11-07_12-24-45.avi
화재가 감지되어 녹화가 시작되었습니다.
녹화가 종료되었습니다.
녹화된 비디오 파일: C:/Users/kelld/fire/Yolov5-Fire-Detection/CCTVvideo/video/recorded_2023-11-07_12-24-45.avi
동영상을 Firebase Storage에 업로드했습니다: recorded_videos/recorded_2023-11-07_12-24-45.avi
